### Import library

In [1]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_5LMD1mtbmi3WmjfQTNtgUQsYSw1g

 


In [3]:
ace.check_session_timeout(s)

14398.697584

## Scoring

In [24]:
def score(raw_data):
    score_data = raw_data[['turnover', 'returns', 'drawdown']]
    score = pd.DataFrame()
    score = 1.3673 + 0.1757*score_data['turnover'] + 0.6150*score_data['returns'] - 1.0816*score_data['drawdown']
    return score

## Alpha Mixing Templates

In [18]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [19]:
'''
signal1 = oth176_lmt_close_eq_score ;
signal2 = oth176_malta_eq_score ;
k = ts_corr(signal1, signal2, 30) ;
k*regression_neut(signal1, signal2)

[아이디어]
경향성이 비슷한 알파 활용하여 corr로 weight를 부여
'''

'\nsignal1 = oth176_lmt_close_eq_score ;\nsignal2 = oth176_malta_eq_score ;\nk = ts_corr(signal1, signal2, 30) ;\nk*regression_neut(signal1, signal2)\n\n[아이디어]\n경향성이 비슷한 알파 활용하여 corr로 weight를 부여\n'

## Read all passed alphas

In [3]:
file_direc = glob.glob('Files/signal/signal_form/*')
file_direc

['Files/signal/signal_form\\A_signalform.csv',
 'Files/signal/signal_form\\B_signalform.csv',
 'Files/signal/signal_form\\C_signalform.csv',
 'Files/signal/signal_form\\D_signalform.csv']

In [4]:
expressions = {}
expressions['A'] = pd.read_csv(file_direc[0])['expression']
expressions['B'] = pd.read_csv(file_direc[1])['expression']
expressions['C'] = pd.read_csv(file_direc[2])['expression']
expressions['D'] = pd.read_csv(file_direc[3])['expression']

## Template A mixing
use top 32 results

In [5]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [12]:
a = expressions['A'][:32]
b = expressions['B'][:24]
c = expressions['C'][:3]
d = expressions['D']
print(f'a = {len(a)}, b = {len(b)}, c = {len(c)}, d = {len(d)}')

a = 32, b = 24, c = 3, d = 28


In [ ]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

### add

In [22]:
alpha_list_AD = []

for i in range(5):
    for j in range(3):
        alpha_list_AD.append(ace.generate_alpha(f'{a[i]};{d[j]};add(scale_down(signal_A_{i}), scale_down(signal_D_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01, decay = 5))

alpha_list_AD

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);WR_D_0 = (high-close)/(high-low) * -100;cond_D_0 = volume > adv20;gr_D_0 = group_rank(cond_D_0, subindustry);signal_D_0 = trade_when(WR_D_0<-80, gr_D_0, WR_D_0 >-20 );add(scale_down(signal_A_0), 0.2*scale_down(signal_D_0), filter = false)'},
 {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay

In [7]:
alpha_list_AC = []

for i in range(10):
    for j in range(3):
        alpha_list_AC.append(ace.generate_alpha(f'{a[i]};{c[j]};add(scale_down(signal_A_{i}), scale_down(signal_C_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01, decay = 5))

alpha_list_AC

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);x_C_0 = regression_neut(implied_volatility_mean_10, implied_volatility_put_10);g_C_0 = bucket(rank(implied_volatility_mean_10), range = "0, 1, 0.1") ;s_C_0 = group_neutralize(x_C_0,g_C_0);signal_C_0 = ts_backfill(ts_decay_linear(s_C_0, 15), 63);add(scale_down(signal_A_0), scale_down(signal_C_0), filter = false)'},
 {'type': 'REGULAR',
  'settings': {'instrumen

In [23]:
result = ace.simulate_alpha_list_multi(s, alpha_list_AD)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [17:42<00:00, 212.44s/it]


In [25]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,12619416,20000000,877,940,0.6402,0.1219,0.0812,0.000381,0.92,2.11,2012-01-22,eOEO72l,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
1,12332406,20000000,882,935,0.6324,0.1191,0.0808,0.000377,0.91,2.09,2012-01-22,WpJpjnZ,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
2,12152685,20000000,886,931,0.6293,0.1173,0.0746,0.000373,0.89,2.05,2012-01-22,6wNwXYO,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
3,13292583,20000000,648,683,0.7166,0.1284,0.0752,0.000358,0.89,2.10,2012-01-22,dxvxgPE,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,FAIL,PASS,FAIL,PASS,PASS,PASS
4,13483560,20000000,645,686,0.7244,0.1302,0.0737,0.000359,0.89,2.10,2012-01-22,17071RK,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,FAIL,PASS,FAIL,PASS,PASS,PASS
5,12665453,20000000,880,937,0.6345,0.1223,0.0952,0.000385,0.89,2.02,2012-01-22,RpPpYAz,pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
6,13170863,20000000,652,679,0.7138,0.1272,0.0768,0.000356,0.88,2.08,2012-01-22,MpVpqvk,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,FAIL,FAIL,FAIL,PASS,PASS,PASS
7,12359834,20000000,882,935,0.6269,0.1193,0.0948,0.000381,0.87,1.99,2012-01-22,AwEwbvQ,pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
8,12166882,20000000,886,931,0.6239,0.1175,0.0890,0.000377,0.85,1.96,2012-01-22,RpPpYEz,pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS
9,11887221,20000000,550,594,0.6385,0.1148,0.0745,0.000360,0.81,1.92,2012-01-22,jZOZ2W5,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,FAIL,FAIL,PASS,PASS,PASS


In [26]:
result_st1.to_csv('AD_mixing_add.csv')

In [26]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,value,year
0,xx5mk9b,NaN,1.58,LOW_SHARPE,PASS,NaN,2.3000,NaN
1,xx5mk9b,NaN,1.00,LOW_FITNESS,PASS,NaN,1.4500,NaN
2,xx5mk9b,NaN,0.01,LOW_TURNOVER,PASS,NaN,0.4059,NaN
3,xx5mk9b,NaN,0.70,HIGH_TURNOVER,PASS,NaN,0.4059,NaN
4,xx5mk9b,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN


In [27]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['RpPk8Yb' 'dxvl07j' 'Lp3ldb1' 'lV1R3gl' 'XpMYRv1']
Passed alphas:['xx5mk9b', 'PpKXOjK', 'ngrO8ad', '37Jne1z', 'lV1R3Je']


In [28]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [29]:
for alpha_id in passed_alphas:
    print(ace.check_prod_corr_test(s, alpha_id))

               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.9  xx5mk9b
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    1.0  PpKXOjK
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.9  ngrO8ad
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.8  37Jne1z
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.8  lV1R3Je


## resimulation

In [42]:
temp = pd.read_csv('A_mixing_add_fin.csv')
temp

,Unnamed: 0,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,0.0,14787979.0,20000000.0,943.0,982.0,0.3519,0.1428,0.0538,0.000812,1.70,2.67,2012-01-22,YpJom7l,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS
1,1.0,14576358.0,20000000.0,935.0,989.0,0.3423,0.1408,0.0578,0.000822,1.60,2.49,2012-01-22,roVMLl3,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS
2,2.0,16908540.0,20000000.0,1029.0,1071.0,0.4707,0.1633,0.0670,0.000694,1.48,2.51,2012-01-22,qA32a9A,pro_A_8 = oth460_es_ebit_fy1_d1m_l1;stock_A_8 ...,PASS,PASS,PASS,PASS,PASS,PASS
3,3.0,17598017.0,20000000.0,808.0,823.0,0.4836,0.1699,0.0791,0.000703,1.46,2.46,2012-01-22,xxW7pdN,pro_A_12 = oth460_1l_m3r_mtn_spb_se;stock_A_12...,PASS,PASS,PASS,PASS,PASS,PASS
4,4.0,15752825.0,20000000.0,925.0,928.0,0.4829,0.1521,0.0636,0.000630,1.45,2.59,2012-01-22,jZxK0AZ,pro_A_6 = oth460_1l_m3r_mtn_spb_se;stock_A_6 =...,PASS,PASS,PASS,PASS,PASS,PASS
5,5.0,15520980.0,20000000.0,935.0,933.0,0.4896,0.1499,0.0740,0.000612,1.45,2.62,2012-01-22,Awm8Y2l,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,PASS,PASS,PASS,PASS,PASS
6,6.0,12149700.0,20000000.0,926.0,966.0,0.3474,0.1173,0.0677,0.000676,1.35,2.33,2012-01-22,ngQG5N3,pro_A_14 = oth460_1l_acda;stock_A_14 = ts_zsco...,PASS,PASS,PASS,PASS,PASS,PASS
7,7.0,14390032.0,20000000.0,1367.0,1441.0,0.3691,0.1390,0.0878,0.000753,1.29,2.10,2012-01-22,076Z2wv,pro_A_16 = oth460_es_ebit_fy1_d1m_l1;stock_A_1...,PASS,PASS,PASS,PASS,PASS,PASS
8,8.0,13539449.0,20000000.0,1318.0,1379.0,0.4359,0.1307,0.0763,0.000600,1.25,2.29,2012-01-22,MpZvEOk,pro_A_10 = oth460_1l_fbds;stock_A_10 = ts_zsco...,PASS,PASS,PASS,PASS,PASS,PASS
9,9.0,14848188.0,20000000.0,841.0,905.0,0.3962,0.1434,0.0916,0.000724,1.24,2.06,2012-01-22,lV9nxJ8,pro_A_18 = oth460_es_roe_fy1_d1m_l1;stock_A_18...,PASS,PASS,PASS,PASS,PASS,PASS


In [43]:
passed_expression = [temp.iloc[i]['expression'] for i in range(13)]

In [44]:
passed_expression

['pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21);factor_A_1 = group_zscore(ts_ir(returns, 252), industry);alpha_A_1 = regression_neut(stock_A_1, factor_A_1);signal_A_1 = trade_when(pro_A_1<=0.5, alpha_A_1, pro_A_1>0.5);zscore(add(signal_A_0, signal_A_1, filter = false))',
 'pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21);factor_A_2 = group_zscore(ts_ir(returns, 252), industry);alpha_A_2 = vector_neut(stock_A_2, factor_A_2);signal_A_2 = trade_when(pro_A_2<=0.5, alpha_A_2, pro_A_2>0.5);pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_3 = group_zscore(ts_ir(returns, 252), industr

In [50]:
alpha_list = [ace.generate_alpha(x, region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.02, decay = 5) for x in passed_expression]

alpha_list

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'MARKET',
   'truncation': 0.02,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21);factor_A_1 = group_zscore(ts_ir(returns, 252), industry);alpha_A_1 = regression_neut(stock_A_1, factor_A_1);signal_A_1 = trade_when(pro_A_1<=0.5, alpha_A_1, pro_A_1>0.5);zscore(add(signal_A_0, signal_A_1, filter = false))'},
 {'type': 'REGULAR',
  'settings': {'inst

In [51]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [26:46<00:00, 321.34s/it]


In [52]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,13148568,20000000,941,984,0.2665,0.1270,0.0522,0.000953,1.65,2.39,2012-01-22,MpZJEKn,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,13116265,20000000,934,991,0.2611,0.1267,0.0557,0.000970,1.57,2.26,2012-01-22,xxWJLel,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
2,14914371,20000000,925,928,0.4040,0.1440,0.0611,0.000713,1.46,2.44,2012-01-22,ngQLAox,pro_A_6 = oth460_1l_m3r_mtn_spb_se;stock_A_6 =...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,14701038,20000000,935,933,0.4092,0.1420,0.0654,0.000694,1.46,2.47,2012-01-22,QagwYlG,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,16601812,20000000,806,825,0.4054,0.1603,0.0783,0.000791,1.45,2.31,2012-01-22,7wMeoq5,pro_A_12 = oth460_1l_m3r_mtn_spb_se;stock_A_12...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,15633768,20000000,1025,1075,0.3874,0.1510,0.0625,0.000779,1.44,2.30,2012-01-22,9wmNeJd,pro_A_8 = oth460_es_ebit_fy1_d1m_l1;stock_A_8 ...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
6,10431152,20000000,923,969,0.2629,0.1007,0.0734,0.000766,1.24,2.00,2012-01-22,GwxZ2G5,pro_A_14 = oth460_1l_acda;stock_A_14 = ts_zsco...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
7,12854034,20000000,1362,1447,0.2862,0.1241,0.0948,0.000867,1.22,1.86,2012-01-22,9wmNeVV,pro_A_16 = oth460_es_ebit_fy1_d1m_l1;stock_A_1...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
8,12216787,20000000,1316,1382,0.3536,0.1180,0.0755,0.000667,1.19,2.06,2012-01-22,3758KXN,pro_A_10 = oth460_1l_fbds;stock_A_10 = ts_zsco...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
9,13017343,20000000,837,909,0.3110,0.1257,0.0909,0.000808,1.14,1.80,2012-01-22,eOvXP1M,pro_A_18 = oth460_es_roe_fy1_d1m_l1;stock_A_18...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS


In [53]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,value,year
0,7wMeoq5,NaN,1.58,LOW_SHARPE,PASS,NaN,2.3100,NaN
1,7wMeoq5,NaN,1.00,LOW_FITNESS,PASS,NaN,1.4500,NaN
2,7wMeoq5,NaN,0.01,LOW_TURNOVER,PASS,NaN,0.4054,NaN
3,7wMeoq5,NaN,0.70,HIGH_TURNOVER,PASS,NaN,0.4054,NaN
4,7wMeoq5,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN


In [54]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['eOvXP1M' 'Vp5ALZM' 'RpKGolz' 'xxWJLel' 'GwxZ2G5' '9wmNeVV' '3758KXN'
 'p63qrqV']
Passed alphas:['ngQLAox', 'QagwYlG', 'MpZJEKn', '7wMeoq5', '9wmNeJd']


In [55]:
for alpha_id in passed_alphas:
    print(ace.check_prod_corr_test(s, alpha_id))

               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.8  ngQLAox
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.8  QagwYlG
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.9  MpZJEKn
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    0.9  7wMeoq5
               test result  limit  value alpha_id
0  PROD_CORRELATION   FAIL    0.7    1.0  9wmNeJd


In [46]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

In [47]:
submit_result

{'xxW7pdN': False,
 'YpJom7l': False,
 'roVMLl3': False,
 'qA32a9A': False,
 'Awm8Y2l': False,
 'jZxK0AZ': False}

In [83]:
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [87]:
ace.get_check_submission(s, passed_alphas[0])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.0200,NaN,oKP2Wrl
1,LOW_FITNESS,PASS,1.00,1.1200,NaN,oKP2Wrl
2,LOW_TURNOVER,PASS,0.01,0.3259,NaN,oKP2Wrl
3,HIGH_TURNOVER,PASS,0.70,0.3259,NaN,oKP2Wrl
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,oKP2Wrl
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.87,0.9200,NaN,oKP2Wrl
6,SELF_CORRELATION,PASS,0.70,0.4739,NaN,oKP2Wrl
7,PROD_CORRELATION,PASS,0.70,0.6252,NaN,oKP2Wrl
8,REGULAR_SUBMISSION,PASS,4.00,3.0000,NaN,oKP2Wrl
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",oKP2Wrl


When you got a list of submittable alphas, you can call function submit_alpha()

In [89]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

In [90]:
#submit_result will have return values from the submit_result function

submit_result

{'oKP2Wrl': True}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
